In [18]:
from settings import leagues, LEAGUE_URL
from library import get_season_years
# ++-------
players = {}


In [19]:
urls = []

for year in range(2010,2022):
    season = get_season_years(year)
    for country in leagues:
        for tier in leagues[country]:
            league_id = leagues[country][tier]["id"]
            url = LEAGUE_URL.format(leagues[country][tier]["id"], season)
            urls.append(f"{url}stats/")


In [20]:
# import json
# import requests
# from time import sleep


# def scrape_players(urls, players):
#     from selenium import webdriver
#     from bs4 import BeautifulSoup
#     options = webdriver.ChromeOptions()
#     failed_urls = []
#     no_nationality = []
#     no_position = []
#     no_yob = []
#     options.add_argument('--headless')
#     options.add_argument('--disable-gpu')
#     driver = webdriver.Chrome(chrome_options=options) # "/usr/lib/chromium-browser/chromedriver")

        
#     for url in urls:
#         response = requests.head(url)
#         if not response.status_code == 200:
#             print(f"{url} does not exist.")
#             print(response.status_code)
#             failed_urls.append(url)
#             continue
#         else:
#             print(f"{'='*20}\nscraping data from {url}\n{'='*20}")

#         # load the page and wait for the table to load
#         driver.get(url)
#         driver.implicitly_wait(10)

#         # get the page source
#         html_document = driver.page_source

#         # parse the page source with BeautifulSoup
#         soup = BeautifulSoup(html_document, 'html.parser')

#         # find the table with id "stats_standard"
#         standard_table = soup.find("table", id="stats_standard")

#         if not standard_table:
#             print(f"{url} without standard table")
#             continue

#         for tr in standard_table.find_all('tr'):
#             tds = tr.find_all('td') 
#             for td in tds:
#                 if not td.has_attr("data-stat"):
#                     print(f"No data stat row in this tr, skipping row")
#                     continue
#                 # Name URL and ID
#                 if td["data-stat"] == "player":
#                     a_tag = td.find("a")
#                     # if not a_tag:
#                     #     print(f"No a_tag for that player, skipping player")
#                     #     continue
#                     player_name = a_tag.string
#                     player_name = player_name.replace(" ", "-")
#                     if player_name in players:
#                         print(f"{player_name} Exists")
#                         continue
#                     player_url = a_tag["href"]
#                     players[player_name] = {"id": player_url.split("/")[3], "url": player_url}
#                 # Nationality 
#                 if td["data-stat"] == "nationality":
#                     a_tag = td.find("a")
#                     if not a_tag:
#                         print(f"No a_tag for {player_name} Nationality, skipping Nationality")
#                         no_nationality.append(player_name)
#                     else:
#                         nationality = a_tag.find("span").find("span")
#                         if not nationality:
#                             print(f"No nationality data for {player_name} Nationality, skipping Nationality")
#                             no_nationality.append(player_name)
#                         else:
#                             nationality = nationality.string
#                             players[player_name]["nationality"] = [nationality]
#                 # Position
#                 if td["data-stat"] == "position":
#                     position = td.string
#                     if not position:
#                         print(f"No position data found for {player_name}, skipping Position")
#                         no_position.append(player_name)
#                     else:
#                         players[player_name]["position"] = [position.split(",")]
                
#                 # Year of Birth
#                 if td["data-stat"] == "birth_year":
#                     birth_year = td.string
#                     if not birth_year:
#                         print(f"No birth_year found for {player_name}, going to next player")
#                         no_yob.append(player_name)
#                     else:
#                         players[player_name]["birth_year"] = int(birth_year)
#     driver.close()
#     return {"players_dict": players, "failed_urls": failed_urls, "no_nationality": no_nationality, "no_position": no_position, "no_yob": no_yob}



In [ ]:
# from settings import players
players = players_info.get("players_dict").copy()
for player_name in players:
    pl_url = players[player_name].get("id")
    if not pl_url:
        continue
    
    id = pl_url.split("/")[3]
    players[player_name]["id"] = id

    if not players[player_name].get("position"):
        continue
    players[player_name]["position"] = players[player_name]["position"][0]

In [ ]:
from settings import players
player_name = "Nabil-Abidallah"
pl_url = players[player_name]["id"]
pl_url.split("/")

In [ ]:
players["Nabil-Abidallah"]["position"][0]

In [ ]:
import json

with open('players.py', 'w') as f:
    json.dump(players, f)


In [ ]:
failed_urls = failed_urls(urls)

In [ ]:
players_info

In [26]:
# TODO: When the response status is 429 only then wait for an hour.

import json
import requests
import time
from typing import Dict


def check_status_code(url: str) -> int:
    response = requests.head(url)
    if response.status_code != 200:
        if response.status_code == 429:
            time.sleep(3600/2)  # wait for half an hour
            response = requests.head(url)
        return response.status_code
    return 200


def scrape_players(
    urls: list, players: Dict, request_interval=30, max_requests=20
) -> Dict:
    from selenium import webdriver
    from bs4 import BeautifulSoup

    options = webdriver.ChromeOptions()
    failed_urls = []
    no_nationality = []
    no_position = []
    no_yob = []
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(
        chrome_options=options
    )  # "/usr/lib/chromium-browser/chromedriver")

    # Do only 20 requests then wait 30s
    request_count = 0
    for url in urls:
        status_code = check_status_code(url)
        if status_code != 200:
            print(f"{url} does not exist. Status code: {status_code}")
            failed_urls.append(url)
            continue
        else:
            print(f"{'='*40}\nscraping data from {url}\n{'='*40}")
            request_count += 1
            if request_count == max_requests:
                time.sleep(request_interval)
                request_count = 0
        try:
            # load the page and wait for the table to load
            driver.get(url)
            driver.implicitly_wait(10)

            # get the page source
            html_document = driver.page_source

            # parse the page source with BeautifulSoup
            soup = BeautifulSoup(html_document, "html.parser")

            # find the table with id "stats_standard"
            standard_table = soup.find("table", id="stats_standard")

            if not standard_table:
                print(f"{url} without standard table")
                continue

            for tr in standard_table.find_all("tr"):
                tds = tr.find_all("td")
                for td in tds:
                    if not td.has_attr("data-stat"):
                        print(f"No data stat row in this tr, skipping row")
                        continue
                    # Name URL and ID
                    if td["data-stat"] == "player":
                        a_tag = td.find("a")
                        player_name = a_tag.string
                        player_name = player_name.replace(" ", "-")
                        if player_name in players:
                            print(f"{player_name} Exists")
                            continue
                        player_url = a_tag["href"]
                        players[player_name] = {
                            "id": player_url.split("/")[3],
                            "url": player_url,
                        }
                    # Nationality
                    if td["data-stat"] == "nationality":
                        a_tag = td.find("a")
                        if not a_tag:
                            print(
                                f"No a_tag for {player_name} Nationality, skipping Nationality"
                            )
                            no_nationality.append(player_name)
                        else:
                            nationality = a_tag.find("span").find("span")
                            if not nationality:
                                print(
                                    f"No nationality data for {player_name} Nationality, skipping Nationality"
                                )
                                no_nationality.append(player_name)
                            else:
                                nationality = nationality.string
                                players[player_name]["nationality"] = [nationality]
                    # Position
                    if td["data-stat"] == "position":
                        position = td.string
                        if not position:
                            print(
                                f"No position data found for {player_name}, skipping Position"
                            )
                            no_position.append(player_name)
                        else:
                            players[player_name]["position"] = [position.split(",")]

                    # Year of Birth
                    if td["data-stat"] == "birth_year":
                        birth_year = td.string
                        if not birth_year:
                            print(
                                f"No birth_year found for {player_name}, going to next player"
                            )
                            no_yob.append(player_name)
                        else:
                            players[player_name]["birth_year"] = int(birth_year)
        except Exception as e:
            print(f"An error occurred while scraping {url}: {e}")
    driver.close()
    return {
        "players_dict": players,
        "failed_urls": failed_urls,
        "no_nationality": no_nationality,
        "no_position": no_position,
        "no_yob": no_yob,
    }


In [27]:
players_info = scrape_players(urls=urls, players={})

/tmp/ipykernel_21534/3853369512.py:30: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


scraping data from https://fbref.com/en/comps/9/2010-2011/stats/
Darren-Bent Exists
David-Bentley Exists
Wayne-Bridge Exists
John-Carew Exists
Andy-Carroll Exists
Eiður-Guðjohnsen Exists
Stephen-Ireland Exists
Gaël-Kakuta Exists
Robbie-Keane Exists
Paul-Konchesky Exists
James-Milner Exists
Steven-Pienaar Exists
Andy-Reid Exists
Paul-Robinson Exists
Roque-Santa-Cruz Exists
Steve-Sidwell Exists
Daniel-Sturridge Exists
Fernando-Torres Exists
Carlos-Vela Exists
Kyle-Walker Exists
scraping data from https://fbref.com/en/comps/10/2010-2011/stats/
Kemy-Agustien Exists
Yakubu-Aiyegbeni Exists
Calvin-Andrew Exists
Ian-Ashbee Exists
Daniel-Ayala Exists
Barry-Bannan Exists
Roman-Bednář Exists
Julian-Bennett Exists
Marcus-Bent Exists
Jens-Berthel-Askou Exists
Ryan-Bertrand Exists
Kris-Boyd Exists
Leon-Britton Exists
Jeffrey-Bruma Exists
Jimmy-Bullard Exists
Stephen-Bywater Exists
DJ-Campbell Exists
Darren-Carter Exists
Pascal-Chimbonda Exists
Leon-Clarke Exists
Jack-Colback Exists
Neill-Collins Ex

In [1]:
players_info

NameError: name 'players_info' is not defined